In [199]:

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV, RandomizedSearchCV
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score, roc_curve, auc, precision_recall_curve
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from scipy.stats import zscore
from scipy.stats import skew, kurtosis
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.decomposition import PCA
%matplotlib inline
from statsmodels.formula.api import ols
import statsmodels.api as sm
from sklearn.linear_model import LinearRegression


In [200]:
df = pd.read_csv('/Users/aimaldastagirzada/Downloads/conversion_data.csv')

In [201]:
df

,Unnamed: 0,visitor_id,visitor_location,marketing_channel,gender,product_views_Mon,product_views_Tue,product_views_Wed,product_views_Thu,product_views_Fri,...,time_on_site_Tue,time_on_site_Wed,time_on_site_Thu,time_on_site_Fri,pages_visited_Mon,pages_visited_Tue,pages_visited_Wed,pages_visited_Thu,pages_visited_Fri,converted_Fri
0,0,1,North America,youtube,M,4,5,2,1,2,...,33,23,46,57,4,5,5,4,6,1
1,1,2,South America,organic,F,2,5,0,1,5,...,28,39,24,51,5,6,7,6,2,0
2,2,3,Australia,instagram,M,3,4,0,-3,5,...,34,22,34,31,1,3,3,2,3,1
3,3,4,North America,facebook,M,1,0,0,5,0,...,26,43,22,39,3,4,7,7,1,0
4,4,5,Asia,instagram,F,1,5,2,2,3,...,54,30,47,57,6,6,5,2,4,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,1995,1996,Europe,organic,F,3,0,0,4,2,...,22,51,44,29,5,1,5,7,7,0
1996,1996,1997,South America,google,M,1,2,0,4,1,...,54,33,59,38,1,6,6,7,6,0
1997,1997,1998,Asia,facebook,M,2,3,0,0,8,...,32,30,48,38,2,1,7,6,4,0
1998,1998,1999,South America,youtube,F,2,2,2,5,5,...,21,51,41,56,6,4,2,2,3,1


In [ ]:
df.drop(columns= 'Unnamed: 0' , axis=1, inplace=True)

In [ ]:
df

In [ ]:
df.info()

In [ ]:
df.columns

In [ ]:
df['avg_product_view'] = df[['product_views_Mon','product_views_Tue','product_views_Wed','product_views_Thu', 'product_views_Fri']].mean(axis=1)
df['avg_rating']= df[['avg_rating_Mon','avg_rating_Tue', 'avg_rating_Wed', 'avg_rating_Thu', 'avg_rating_Fri',]].mean(axis=1)
df['avg_price']=df[['avg_price_Mon', 'avg_price_Tue', 'avg_price_Wed', 'avg_price_Thu','avg_price_Fri']].mean(axis=1)
df['avg_time_spent']= (df['time_on_site_Mon']+df['time_on_site_Tue']+df['time_on_site_Wed']+df['time_on_site_Thu']+df['time_on_site_Fri'])/5
df['avg_pages_visted'] = (df['pages_visited_Mon']+df['pages_visited_Tue']+df['pages_visited_Wed']+df['pages_visited_Thu']+df['pages_visited_Fri'])/5

In [ ]:
df

In [ ]:
fig, axs=plt.subplots(2,3, figsize=(15,10))
plt1= sns.boxplot(df['avg_product_view'], ax=axs[0][0])
plt2= sns.boxplot(df['avg_rating'], ax=axs[0][1])
plt3= sns.boxplot(df['avg_price'], ax=axs[0][2])
plt4= sns.boxplot(df['avg_time_spent'], ax=axs[1][0])
plt5= sns.boxplot(df['avg_pages_visted'], ax=axs[1][1])

In [ ]:
df.columns

In [ ]:
numerical_column= ['avg_product_view','avg_rating', 'avg_price', 'avg_time_spent','avg_pages_visted']
categorical_column= ['visitor_location', 'marketing_channel','gender','converted_Fri']


In [ ]:
for column in numerical_column:
    skewness_values = []
    s = skew(df[column])
    k = kurtosis(df[column])
    print(f'{column}: skewness = {s}, kurtosis = {k}')

In [ ]:
df2=df.copy()

In [ ]:
if abs(s) > 1:
    df2[column] = df2[column].apply(lambda x: np.log(x + 1))
z_scores = zscore(df2[numerical_column])
threshold = 3
df2 = df2[(np.abs(z_scores) < threshold).all(axis=1)]

In [ ]:
df.info()

In [ ]:
df2.info()

In [ ]:
if abs(s) > 1:
    df[column] = df[column].apply(lambda x: np.log(x + 1))
z_scores = zscore(df[numerical_column])
threshold = 3
df = df[(np.abs(z_scores) < threshold).all(axis=1)]

In [ ]:
fig, axs=plt.subplots(2,3, figsize=(15,10))
plt1= sns.boxplot(df['avg_product_view'], ax=axs[0][0])
plt2= sns.boxplot(df['avg_rating'], ax=axs[0][1])
plt3= sns.boxplot(df['avg_price'], ax=axs[0][2])
plt4= sns.boxplot(df['avg_time_spent'], ax=axs[1][0])
plt5= sns.boxplot(df['avg_pages_visted'], ax=axs[1][1])

In [ ]:
df.head()

In [ ]:
df.columns

In [ ]:
df.head()


**Statistical Testing**


In [ ]:
df.groupby('visitor_location')[['avg_product_view', 'avg_rating', 'avg_price', 'avg_time_spent',
       'avg_pages_visted']].mean()

In [ ]:
loc1=df[df['visitor_location']=='Asia']['avg_product_view']
loc2=df[df['visitor_location']=='Australia']['avg_product_view']
loc3=df[df['visitor_location']=='Europe']['avg_product_view']
loc4=df[df['visitor_location']=='North America']['avg_product_view']
loc5=df[df['visitor_location']=='South America']['avg_product_view']

p_val= stats.f_oneway(loc1,loc2,loc3,loc4,loc5)
print(p_val)



In [ ]:
loc1=df[df['visitor_location']=='Asia']['avg_rating']
loc2=df[df['visitor_location']=='Australia']['avg_rating']
loc3=df[df['visitor_location']=='Europe']['avg_rating']
loc4=df[df['visitor_location']=='North America']['avg_rating']
loc5=df[df['visitor_location']=='South America']['avg_rating']

p_val=stats.f_oneway(loc1,loc2,loc3,loc4,loc5)
print(p_val)

In [ ]:
loc1=df[df['visitor_location']=='Asia']['avg_price']
loc2=df[df['visitor_location']=='Australia']['avg_price']
loc3=df[df['visitor_location']=='Europe']['avg_price']
loc4= df[df['visitor_location']=='North America']['avg_price']
loc5=df[df['visitor_location']=='South America']['avg_price']

p_val=stats.f_oneway(loc1,loc2,loc3,loc4,loc5)
print(p_val)

In [ ]:
loc1=df[df['visitor_location']=='Asia']['avg_time_spent']
loc2=df[df['visitor_location']=='Australia']['avg_time_spent']
loc3=df[df['visitor_location']=='Europe']['avg_time_spent']
loc4= df[df['visitor_location']=='North America']['avg_time_spent']
loc5=df[df['visitor_location']=='South America']['avg_time_spent']
p_val=stats.f_oneway(loc1,loc2,loc3,loc4,loc5)
print(p_val)


In [ ]:
loc1=df[df['visitor_location']=='Asia']['avg_pages_visted']
loc2=df[df['visitor_location']=='Australia']['avg_pages_visted']
loc3=df[df['visitor_location']=='Europe']['avg_pages_visted']
loc4= df[df['visitor_location']=='North America']['avg_pages_visted']
loc5=df[df['visitor_location']=='South America']['avg_pages_visted']
p_val=stats.f_oneway(loc1,loc2,loc3,loc4,loc5)
print(p_val)

**Avg pages visited has signifcant impact**

In [ ]:
df.groupby('marketing_channel')[['avg_product_view', 'avg_rating', 'avg_price', 'avg_time_spent','avg_pages_visted']].mean()

In [ ]:
m1=df[df['marketing_channel']=='youtube']['avg_product_view']
m2=df[df['marketing_channel']=='organic']['avg_product_view']
m3=df[df['marketing_channel']=='instagram']['avg_product_view']
m4=df[df['marketing_channel']=='facebook']['avg_product_view']
m5=df[df['marketing_channel']=='google']['avg_product_view']

p_val= stats.f_oneway(m1,m2,m3,m4,m5)
print(p_val)

In [ ]:
m1=df[df['marketing_channel']=='youtube']['avg_rating']
m2=df[df['marketing_channel']=='organic']['avg_rating']
m3=df[df['marketing_channel']=='instagram']['avg_rating']
m4=df[df['marketing_channel']=='facebook']['avg_rating']
m5=df[df['marketing_channel']=='google']['avg_rating']

p_val= stats.f_oneway(m1,m2,m3,m4,m5)
print(p_val)

In [ ]:
m1=df[df['marketing_channel']=='youtube']['avg_price']
m2=df[df['marketing_channel']=='organic']['avg_price']
m3=df[df['marketing_channel']=='instagram']['avg_price']
m4=df[df['marketing_channel']=='facebook']['avg_price']
m5=df[df['marketing_channel']=='google']['avg_price']

p_val= stats.f_oneway(m1,m2,m3,m4,m5)
print(p_val)

In [ ]:
m1=df[df['marketing_channel']=='youtube']['avg_time_spent']
m2=df[df['marketing_channel']=='organic']['avg_time_spent']
m3=df[df['marketing_channel']=='instagram']['avg_time_spent']
m4=df[df['marketing_channel']=='facebook']['avg_time_spent']
m5=df[df['marketing_channel']=='google']['avg_time_spent']

p_val= stats.f_oneway(m1,m2,m3,m4,m5)
print(p_val)

In [ ]:
m1=df[df['marketing_channel']=='youtube']['avg_pages_visted']
m2=df[df['marketing_channel']=='organic']['avg_pages_visted']
m3=df[df['marketing_channel']=='instagram']['avg_pages_visted']
m4=df[df['marketing_channel']=='facebook']['avg_pages_visted']
m5=df[df['marketing_channel']=='google']['avg_pages_visted']

p_val= stats.f_oneway(m1,m2,m3,m4,m5)
print(p_val)

In [ ]:
df.groupby('gender')[['avg_product_view', 'avg_rating', 'avg_price', 'avg_time_spent','avg_pages_visted']].mean()

In [ ]:
g1= df[df['gender']=='F']['avg_product_view']
g2= df[df['gender']=='M']['avg_product_view']

p_val=stats.f_oneway(g1,g2)
print(p_val)

In [ ]:
g1=df[df['gender']=='F']['avg_rating']
g2=df[df['gender']=='M']['avg_rating']

p_val=stats.f_oneway(g1,g2)
print(p_val)

In [ ]:
g1=df[df['gender']=='F']['avg_price']
g2=df[df['gender']=='M']['avg_price']

p_val=stats.f_oneway(g1,g2)
print(p_val)

In [ ]:
g1=df[df['gender']=='F']['avg_time_spent']
g2=df[df['gender']=='M']['avg_time_spent']

p_val=stats.f_oneway(g1,g2)
print(p_val)

In [ ]:
g1=df[df['gender']=='F']['avg_pages_visted']
g2=df[df['gender']=='M']['avg_pages_visted']

p_val=stats.f_oneway(g1,g2)
print(p_val)

**There is a statiscally significant difference Avg pages visited **
****

In [ ]:
df.columns

In [ ]:
all_numerical_cols = ['product_views_Mon', 'product_views_Tue', 'product_views_Wed',
       'product_views_Thu', 'product_views_Fri', 'avg_rating_Mon',
       'avg_rating_Tue', 'avg_rating_Wed', 'avg_rating_Thu', 'avg_rating_Fri',
       'avg_price_Mon', 'avg_price_Tue', 'avg_price_Wed', 'avg_price_Thu',
       'avg_price_Fri', 'time_on_site_Mon', 'time_on_site_Tue',
       'time_on_site_Wed', 'time_on_site_Thu', 'time_on_site_Fri',
       'pages_visited_Mon', 'pages_visited_Tue', 'pages_visited_Wed',
       'pages_visited_Thu', 'pages_visited_Fri',
       'avg_product_view', 'avg_rating', 'avg_price',
       'avg_time_spent', 'avg_pages_visted']

In [ ]:
for column in all_numerical_cols:
    plt.figure(figsize=(5, 5)) # Optional: Set the figure size
    sns.histplot(df[column], bins=20, kde=True) # You can customize the number of bins, KDE line, etc.
    plt.title(f'Histogram of {column}') # Add a title for each plot
    plt.show()

In [ ]:
df.columns

In [ ]:
relevant_columns= ['visitor_location', 'marketing_channel', 'gender', 'product_views_Wed', 'product_views_Thu', 'product_views_Fri','avg_product_view','avg_rating', 'avg_price', 'avg_time_spent','avg_pages_visted','converted_Fri']

In [ ]:
new_df= df[relevant_columns]

In [ ]:
new_df

In [ ]:
sns.countplot(x='converted_Fri', data=new_df)
plt.show()

sns.pairplot(new_df, hue='converted_Fri')
plt.show()


In [ ]:
new_df.columns

In [ ]:
fig, axs=plt.subplots(3,3, figsize=(15,10))
plt1=sns.histplot(new_df['avg_product_view'], ax=axs[0][0])
plt2=sns.histplot(new_df['avg_rating'], ax=axs[0][1])
plt3= sns.histplot(new_df['avg_price'], ax=axs[0][2])
plt4=sns.histplot(new_df['avg_time_spent'], ax=axs[1][0])
plt5=sns.histplot(new_df['avg_pages_visted'], ax=axs[1][1])
plt6=sns.histplot(new_df['product_views_Wed'], ax=axs[1][2])
plt7=sns.histplot(new_df['product_views_Thu'], ax=axs[2][0])
plt8=sns.histplot(new_df['product_views_Fri'], ax=axs[2][1])


In [ ]:
fig, axs=plt.subplots(3,3, figsize=(15,10))
sns.boxplot(y='avg_product_view', x='converted_Fri', data=new_df, ax=axs[0][0])
sns.boxplot(y='avg_rating', x='converted_Fri', data=new_df, ax=axs[0][1])
sns.boxplot(y='avg_price', x='converted_Fri', data=new_df, ax=axs[0][2])
sns.boxplot(y='avg_time_spent', x='converted_Fri', data=new_df, ax=axs[1][0])
sns.boxplot(y='avg_pages_visted', x='converted_Fri', data=new_df, ax=axs[1][1])
sns.boxplot(y='product_views_Wed', x='converted_Fri', data=new_df, ax=axs[1][2])
sns.boxplot(y='product_views_Thu', x='converted_Fri', data=new_df, ax=axs[2][0])
sns.boxplot(y='product_views_Fri', x='converted_Fri', data=new_df, ax=axs[2][1])



**Inital Hypothesis Product view on Wed,Thursday, Friday and Average product view are stong predictors of Converstion on friday**

In [ ]:
all_numerical_cols

In [ ]:
fig, axs = plt.subplots(6,5, figsize=(25,10))  
sns.boxplot(y='product_views_Mon', x='converted_Fri', data=df,ax=axs[0][0])
sns.boxplot(y='product_views_Tue', x='converted_Fri', data=df,ax=axs[0][1])
sns.boxplot(y='product_views_Wed', x='converted_Fri', data=df,ax=axs[0][2])
sns.boxplot(y='product_views_Thu', x='converted_Fri', data=df,ax=axs[0][3])
sns.boxplot(y='product_views_Fri', x='converted_Fri', data=df,ax=axs[0][4])
sns.boxplot(y='avg_rating_Mon', x='converted_Fri', data=df,ax=axs[1][0])
sns.boxplot(y='avg_rating_Tue', x='converted_Fri', data=df,ax=axs[1][1])
sns.boxplot(y='avg_rating_Wed', x='converted_Fri', data=df,ax=axs[1][2])
sns.boxplot(y='avg_rating_Thu', x='converted_Fri', data=df,ax=axs[1][3])
sns.boxplot(y='avg_rating_Fri', x='converted_Fri', data=df,ax=axs[1][4])
sns.boxplot(y='avg_price_Mon', x='converted_Fri', data=df, ax=axs[2][0])
sns.boxplot(y='avg_price_Tue', x='converted_Fri', data=df,ax=axs[2][1])
sns.boxplot(y='avg_price_Wed', x='converted_Fri', data=df,ax=axs[2][2])
sns.boxplot(y='avg_price_Thu', x='converted_Fri', data=df,ax=axs[2][3])
sns.boxplot(y='avg_price_Fri', x='converted_Fri', data=df,ax=axs[2][4])
sns.boxplot(y='time_on_site_Mon', x='converted_Fri', data=df, ax=axs[3][0])
sns.boxplot(y='time_on_site_Tue', x='converted_Fri', data=df,ax=axs[3][1])
sns.boxplot(y='time_on_site_Wed', x='converted_Fri', data=df,ax=axs[3][2])
sns.boxplot(y='time_on_site_Thu', x='converted_Fri', data=df,ax=axs[3][3])
sns.boxplot(y='time_on_site_Fri', x='converted_Fri', data=df,ax=axs[3][4])
sns.boxplot(y='pages_visited_Mon', x='converted_Fri', data=df, ax=axs[4][0])
sns.boxplot(y='pages_visited_Tue', x='converted_Fri', data=df,ax=axs[4][1])
sns.boxplot(y='pages_visited_Wed', x='converted_Fri', data=df,ax=axs[4][2])
sns.boxplot(y='pages_visited_Thu', x='converted_Fri', data=df,ax=axs[4][3])
sns.boxplot(y='pages_visited_Fri', x='converted_Fri', data=df,ax=axs[4][4])
sns.boxplot(y='avg_price', x='converted_Fri', data=df, ax=axs[5][0])
sns.boxplot(y='avg_time_spent', x='converted_Fri', data=df, ax=axs[5][1])
sns.boxplot(y='avg_pages_visted', x='converted_Fri', data=df, ax=axs[5][2])
sns.boxplot(y='avg_product_view', x='converted_Fri', data=df,ax=axs[5][3])
sns.boxplot(y='avg_rating', x='converted_Fri', data=df, ax=axs[5][4])

In [ ]:
categorical_column

In [ ]:
new_df.groupby(['visitor_location','converted_Fri'])['gender'].count().unstack()

In [ ]:
new_df.groupby(['marketing_channel', 'converted_Fri'])['gender'].count().unstack()

In [ ]:
df.groupby(['gender','converted_Fri'])['gender'].count().unstack()

In [ ]:
new_df[new_df['converted_Fri']==0].describe(include='all')


In [ ]:
new_df[new_df['converted_Fri']==1].describe(include='all')

#MODEL DEVELOPMENT:

In [ ]:
new_df

In [ ]:
numerical_column

In [ ]:
numerical_column.append('product_views_Wed')
numerical_column.append('product_views_Thu')
numerical_column.append('product_views_Fri')

In [ ]:
cat_column= ['visitor_location', 'marketing_channel', 'gender']

In [ ]:
target= 'converted_Fri'

In [ ]:
cat_transformer = OneHotEncoder(handle_unknown='ignore')
num_transformer = StandardScaler()
preprocessor = ColumnTransformer(transformers = [('cat',cat_transformer, cat_column),
                                                 ('num', num_transformer, numerical_column)])

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(new_df[cat_column + numerical_column], new_df[target], test_size=0.2, random_state=123)


In [ ]:
x_train_transformed = preprocessor.fit_transform(X_train)
x_test_transformed = preprocessor.fit_transform(X_test)

In [ ]:
x_train_transformed.shape

In [ ]:
x_test_transformed.shape

In [ ]:
encoded_columns = preprocessor.named_transformers_['cat'].get_feature_names_out(cat_column)

In [ ]:
encoded_columns = encoded_columns.tolist()

In [ ]:
all_feat = numerical_column + encoded_columns

X_train_transformed = pd.DataFrame(x_train_transformed, columns=all_feat)
X_test_transformed = pd.DataFrame(x_test_transformed, columns=all_feat)

In [ ]:
log=LogisticRegression()
nb=GaussianNB()
knn= KNeighborsClassifier()
svc=SVC()
dt= DecisionTreeClassifier()

In [ ]:
log.fit(X_train_transformed, y_train)
nb.fit(X_train_transformed, y_train)
knn.fit(X_train_transformed, y_train)
svc.fit(X_train_transformed, y_train)
dt.fit(X_train_transformed, y_train)


In [ ]:
pred_l= log.predict(X_test_transformed)
pred_nb=nb.predict(X_test_transformed)
pred_knn=knn.predict(X_test_transformed)
pred_svc= svc.predict(X_test_transformed)
pred_dt= dt.predict(X_test_transformed)

In [ ]:
print(f1_score(pred_l,y_test))
print(f1_score(pred_nb,y_test))
print(f1_score(pred_knn,y_test))
print(f1_score(pred_svc,y_test))
print(f1_score(pred_dt,y_test))

In [ ]:
print(accuracy_score(pred_l,y_test))
print(accuracy_score(pred_nb,y_test))
print(accuracy_score(pred_knn,y_test))
print(accuracy_score(pred_svc,y_test))
print(accuracy_score(pred_dt,y_test))

In [ ]:
print(recall_score(pred_l,y_test))
print(recall_score(pred_nb,y_test))
print(recall_score(pred_knn,y_test))
print(recall_score(pred_svc,y_test))
print(recall_score(pred_dt,y_test))

Since we can accept some False positives in this situation, lets pick Recall as our metric. 
# Part 4: Hyperparameter Tuning and Cross Validation

In [ ]:
#Logistics
param_grid = {
    'C': [0.001, 0.01, 0.1, 1, 10, 100, 1000],
    'penalty': ['l1', 'l2'],
    'solver': ['liblinear', 'saga'],
    'class_weight': [None, 'balanced'],
    'max_iter': [50, 100, 200]
}
rs_log = RandomizedSearchCV(log, param_distributions=param_grid, cv=10)
rs_log.fit(x_train_transformed,y_train)
rs_log.best_estimator_

In [ ]:
log_best = LogisticRegression(C=1000,max_iter=50,penalty='l1',solver='saga')

log_best.fit(x_train_transformed,y_train)
pred_best_log = log_best.predict(x_test_transformed)

cv_scores = cross_val_score(log_best, x_train_transformed, y_train, cv=10, scoring='f1')

print("Cross-validation scores for Logistic Regression: ", cv_scores)

In [ ]:
cv_scores.mean()

In [ ]:
print(recall_score(pred_best_log, y_test))

In [ ]:
svc_params = {'kernel':['linear','poly','rbf']}

rs_svc = RandomizedSearchCV(svc, svc_params, n_iter=3, cv=5, scoring='recall')
rs_svc.fit(X_train_transformed, y_train)

In [ ]:
rs_svc.best_estimator_

In [ ]:
svc_best = SVC()
svc_best.fit(X_train_transformed,y_train)
pred_svc_best = svc_best.predict(X_test_transformed)
print(recall_score(pred_svc_best, y_test))

In [ ]:
cv_scores = cross_val_score(svc_best, x_train_transformed, y_train, cv=10, scoring='f1')
print("Cross-validation scores for SVC: ", cv_scores)

In [ ]:
cv_scores.mean()

In [ ]:
#DT
dt = DecisionTreeClassifier(random_state=123)
dt = dt.fit(X_train_transformed, y_train)